# Analysis

In [1]:
import os
import bokeh
from bokeh.plotting import show
import pandas as pd
import numpy as np
import torch

import flowkit as fk
import seaborn as sns
import matplotlib.pyplot as plt

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#load in patient dataset
flow_data = pd.read_csv("flow_data.csv")
flow_data

,Subject Accession,H1,H3,B,FCS Raw,FCS Preprocessed,WSP File
0,SUB120420,64,16,16,Panel_L1_A1_L1_110194_PBMC_10112011_A01.580227...,export_Panel_L1_A1_L1_110194_PBMC_10112011_A01...,Panel_L1_Run_2.804556.wsp
1,SUB120423,256,16,16,Panel_L1_A2_L1_110197_PBMC_10112011_A02.579447...,export_Panel_L1_A2_L1_110197_PBMC_10112011_A02...,Panel_L1_Run_1.804538.wsp
2,SUB120445,64,8,32,Panel_L1_A3_L1_110243_PBMC_10172011_A03.578999...,export_Panel_L1_A3_L1_110243_PBMC_10172011_A03...,Panel_L1_Run_4.804574.wsp
3,SUB120446,1024,256,64,Panel_L1_A1_L1_110244_PBMC_10172011_A01.579604...,export_Panel_L1_A1_L1_110244_PBMC_10172011_A01...,Panel_L1_Run_5.804583.wsp
4,SUB120449,1024,128,64,Panel_L1_E3_L1_110247_PBMC_10172011_E03.580335...,export_Panel_L1_E3_L1_110247_PBMC_10172011_E03...,Panel_L1_Run_4.804574.wsp
5,SUB120450,64,32,16,Panel_L1_E1_L1_110248_PBMC_10172011_E01.578572...,export_Panel_L1_E1_L1_110248_PBMC_10172011_E01...,Panel_L1_Run_5.804583.wsp
6,SUB120452,128,128,128,Panel_L1_A2_L1_110250_PBMC_10172011_A02.579307...,export_Panel_L1_A2_L1_110250_PBMC_10172011_A02...,Panel_L1_Run_5.804583.wsp
7,SUB120457,128,128,32,Panel_L1_E2_L1_110255_PBMC_10172011_E02.580138...,export_Panel_L1_E2_L1_110255_PBMC_10172011_E02...,Panel_L1_Run_5.804583.wsp
8,SUB120458,256,64,32,Panel_L1_A3_L1_110256_PBMC_10172011_A03.579670...,export_Panel_L1_A3_L1_110256_PBMC_10172011_A03...,Panel_L1_Run_5.804583.wsp
9,SUB120459,128,32,64,Panel_L1_A1_L1_110257_PBMC_10172011_A01.580299...,export_Panel_L1_A1_L1_110257_PBMC_10172011_A01...,Panel_L1_Run_6.804592.wsp


In [3]:
cytof_data = pd.read_csv("cytof_data.csv")
cytof_data

,Subject Accession,H1,H3,B,FCS Raw,FCS Preprocessed
0,SUB116519,640,640,160,083012-Mike-Study 15-2011-V1-15103_cells_found...,083012-Mike-Study 15-2011-V1-15103_cells_found...
1,SUB116523,40,160,80,083012-Mike-Study 15-2011-V1-15107_cells_found...,083012-Mike-Study 15-2011-V1-15107_cells_found...
2,SUB116524,160,40,320,083012-Mike-Study 15-2011-V1-15108_cells_found...,083012-Mike-Study 15-2011-V1-15108_cells_found...
3,SUB116525,80,320,160,083012-Mike-Study 15-2011-V1-15109_cells_found...,083012-Mike-Study 15-2011-V1-15109_cells_found...
4,SUB116526,20,160,20,083012-Mike-Study 15-2011-V1-15110_cells_found...,083012-Mike-Study 15-2011-V1-15110_cells_found...
5,SUB116527,320,80,160,083012-Mike-Study 15-2011-V1-15111_cells_found...,083012-Mike-Study 15-2011-V1-15111_cells_found...
6,SUB116528,160,160,320,083012-Mike-Study 15-2011-V1-15112_cells_found...,083012-Mike-Study 15-2011-V1-15112_cells_found...
7,SUB116529,80,320,320,083012-Mike-Study 15-2011-V1-15113_cells_found...,083012-Mike-Study 15-2011-V1-15113_cells_found...
8,SUB116530,160,80,160,083012-Mike-Study 15-2011-V1-15114_cells_found...,083012-Mike-Study 15-2011-V1-15114_cells_found...
9,SUB116531,160,320,160,083012-Mike-Study 15-2011-V1-15115_cells_found...,083012-Mike-Study 15-2011-V1-15115_cells_found...


In [4]:
#Sample Extraction Function
def extractFCSDF(fileName, set, source = "Raw"):
    #Set: CyTOF or Flow
    path = ""
    if source == "Preprocessed":
        path = os.path.join("FCS", set, "Preprocessed", fileName)
    else:
        path = os.path.join("FCS", set, "Raw", fileName)
    sample = fk.Sample(path)
    return sample.as_dataframe(source = "raw")